Data Privacy Final Project 

In [39]:
# Load the data and libraries
import pandas as pd
import numpy as np
import random
from scipy import stats
import matplotlib.pyplot as plt

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def laplace_mech_vec(vec, sensitivity, epsilon):
    return [v + np.random.laplace(loc=0, scale=sensitivity / epsilon) for v in vec]

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

math = pd.read_csv("student_math_clean.csv")
math.head(5)

,student_id,school,sex,age,address_type,family_size,parent_status,mother_education,father_education,mother_job,...,family_relationship,free_time,social,weekday_alcohol,weekend_alcohol,health,absences,grade_1,grade_2,final_grade
0,1,GP,F,18,Urban,Greater than 3,Apart,higher education,higher education,at_home,...,4,3,4,1,1,3,6,5,6,6
1,2,GP,F,17,Urban,Greater than 3,Living together,primary education (4th grade),primary education (4th grade),at_home,...,5,3,3,1,1,3,4,5,5,6
2,3,GP,F,15,Urban,Less than or equal to 3,Living together,primary education (4th grade),primary education (4th grade),at_home,...,4,3,2,2,3,3,10,7,8,10
3,4,GP,F,15,Urban,Greater than 3,Living together,higher education,5th to 9th grade,health,...,3,2,2,1,1,5,2,15,14,15
4,5,GP,F,16,Urban,Greater than 3,Living together,secondary education,secondary education,other,...,4,3,2,1,2,5,4,6,10,10


KeyError: False

In [38]:
## Cache the sorted ages, because we will use them a lot.
absences_lower = 0
absences_upper = 20
sorted_absences = math['absences'].clip(lower=absences_lower, upper=absences_upper).sort_values()

def min_absences():
    clipped_absences = math['absences'].clip(lower=0, upper=100)
    return clipped_absences.min()

def max_absences():
    clipped_absences = math['absences'].clip(lower=0, upper=100)
    return clipped_absences.max()

def ls_min():
    return max(sorted_absences.iloc[0] - absences_lower, sorted_absences.iloc[1] - sorted_absences.iloc[0])

print('Actual minimum absences:', min_absences())
print('Local sensitivity of the minimum:', ls_min())
print('max', max_absences())

Actual minimum absences: 0
Local sensitivity of the minimum: 0
max 75


In [40]:
# preserves epsilon-differential privacy
def above_threshold(query_results, T, epsilon):
    T_hat = T + np.random.laplace(loc=0, scale = 2/epsilon)
    
    for idx, q in enumerate(query_results):
        nu_i = np.random.laplace(loc=0, scale = 4/epsilon)
        if q + nu_i >= T_hat:
            return idx
    return None

In [41]:
def above_15(query_results, epsilon):
    ### BEGIN SOLUTION
    T = 15
    idx = above_threshold(query_results, T, epsilon/2)
    val = laplace_mech(query_results[idx], 1, epsilon/2)
    return val
    ### END SOLUTION

queries = math['absences'].value_counts()
print(f"above_15 #1: {above_15(queries, 100)}")
print(f"above_15 #2: {above_15(queries, 1)}")
print(f"above_15 #3: {above_15(queries, .01)}")

above_15 #1: 114.98664695717343
above_15 #2: 118.21332108137582
above_15 #3: 136.77098425995135


In [43]:
def bounded_all_above_15(query_results, c, epsilon):
    ### BEGIN SOLUTION
    answers = []
    pos = 0
    T = 15
    epsilon_i = epsilon / (c*2)
    
    while len(answers) < c:
        answer_idx = above_threshold(query_results[pos:], T, epsilon_i)
        if answer_idx == None:
            break
        pos = pos + answer_idx
        answers.append(laplace_mech(query_results[pos], 1, epsilon_i))
        pos = pos + 1
        
    return answers
    ### END SOLUTION

queries = list(math['absences'].value_counts())
print(f"bounded_all_above_10000 #1: {bounded_all_above_15(queries, 3, 100)}")
print(f"bounded_all_above_10000 #2: {bounded_all_above_15(queries, 3, 1)}")
print(f"bounded_all_above_10000 #3: {bounded_all_above_10000(queries, 3, .01)}")

NameError: name 'bounded_all_above_10000' is not defined